# Loading the data

In [12]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [13]:
# Getting the data
from pathlib import Path
train_dir = Path('/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/train')

In [ ]:
# Defined the batch size here
batch_size = 32
# Image size 336 is used here because RandomCrop is used later to get a 224*224 part out
img_height = 336
img_width = 336

# Carrying out the train test split
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  label_mode='categorical',
  validation_split=0.2,
  subset='training',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  label_mode='categorical',
  validation_split=0.2,
  subset='validation',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)


In [ ]:
df=pd.read_csv('/kaggle/input/embeddings_matrix2.csv')

col_names=[]
col_names.append("Classes")
for i in range (1,86):
    col_names.append(f'dim{i}')
df.columns=col_names

df_columns = [f"dim{i}" for i in range(1, 86)]
df['embedding'] = df[df_columns].values.tolist()
for i in range(1,86):
    df=df.drop(f'dim{i}',axis=1)
    
attribute_embs=df.set_index('Classes')['embedding'].to_dict()

In [ ]:
labels=train_ds.class_names
labelled_classes = {index: name for index, name in enumerate(labels)}

In [ ]:
def embedding_vectors(image, label):
    # Converting labels to embedding vectors
    class_name = labelled_classes[label.numpy()] 
    embedding_vector = attribute_embs[class_name]
    
    # Converting it to a tensor
    embedding_vector = tf.convert_to_tensor(embedding_vector, dtype=tf.float32)
    
    return image, embedding_vector



def embedding_vectors_with_shape(image, label):
    image, embedding_vector = tf.py_function(
        func=embedding_vectors,
        inp=[image, label],
        Tout=(tf.float32, tf.float32)
    )

    image.set_shape([img_height, img_width, 3])  
    embedding_vector.set_shape([85])
    return image, embedding_vector 
    
train_ds = train_ds.map(embedding_vectors_with_shape)
val_ds = val_ds.map(embedding_vectors_with_shape)


# Creating the model

In [ ]:
# Initializing the model using sequential API
model=tf.keras.models.Sequential()

In [4]:
# Implementing the data augmentation and pixel value rescaling layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),  
    tf.keras.layers.RandomCrop(224, 224),
    tf.keras.layers.Resizing(224, 224),
    tf.keras.layers.Rescaling(1.0/255)      
])

In [ ]:
# Using MobileNetV2 for feature extraction
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet",
    pooling='avg'
)

# Freezing the pre-trained layers
pretrained_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
model.add(data_augmentation)
model.add(pretrained_model)
# Adding layers for personalized task
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(40, activation='softmax'))

In [9]:
model.compile(
    optimizer='Adam',  # Optimizer
    loss=tf.keras.losses.MeanSquaredError,  # Loss function
    metrics=['accuracy']  
)

# Training and fine tuning

Training

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    batch_size=batch_size
)

Fine tuning

In [ ]:
# Unfreezing the pretrained layers
pretrained_model.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.fit(train_ds, epochs=10, validation_data=val_ds)

# Creating the submission csv

In [ ]:
class_names = ['antelope', 'bat', 'beaver', 'blue+whale', 'bobcat', 'buffalo', 'chihuahua', 'chimpanzee', 'collie', 'cow', 'dalmatian', 'deer', 'dolphin', 'elephant', 'fox', 'german+shepherd', 'giant+panda', 'giraffe', 'gorilla', 'grizzly+bear', 'hamster', 'hippopotamus', 'horse', 'humpback+whale', 'killer+whale', 'leopard', 'lion', 'mole', 'moose', 'mouse', 'otter', 'ox', 'persian+cat', 'pig', 'polar+bear', 'rabbit', 'raccoon', 'rat', 'rhinoceros', 'seal', 'sheep', 'siamese+cat', 'skunk', 'spider+monkey', 'squirrel', 'tiger', 'walrus', 'weasel', 'wolf', 'zebra']


In [ ]:
from scipy.spatial.distance import cosine

# Assuming pred_attribs is the predicted vector for a test image
# and class_attributes is the attribute vector for each class (both seen and unseen)
def predict_class(image_batch,model,attribute_embeddings):
    
    similarities = {}
    pred_attribs=model.predict(image_batch)
    batch_pred=[]
    for attrib in pred_attribs:
        for class_name, class_attributes in attribute_embeddings.items():
            similarity = cosine(attrib, class_attributes)
            similarities[class_name] = similarity
            
        # Sort by similarity
        sorted_classes = sorted(similarities, key=similarities.get)
        batch_pred.append(sorted_classes[0]) # Most similar class
    
    return  (batch_pred)


In [49]:
import os
import pandas as pd


# Path to the test dataset
test_dir = '/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/test'

# List of image file paths (assuming all images are in the same directory)
image_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith('.jpg')]

# Function to load and preprocess images
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (336, 336))  
    return image

# Load and preprocess
images = [load_image(path) for path in image_paths]

batch_size = 32
image_dataset = tf.data.Dataset.from_tensor_slices(images)
image_dataset = image_dataset.batch(batch_size)


predictions = []

# For all images in batch
for image_batch in image_dataset:
    predictions.extend(predict_class(image_batch,model,attribute_embs))


# Preparing csv
results = {
    'image': [os.path.basename(path) for path in image_paths],
    'label': predictions
}

# Create a DataFrame and save to CSV
df = pd.DataFrame(results)
df.columns=['image_id','class']
df=df.sort_values('image_id')
df.to_csv('submission.csv', index=False)

print("Predictions saved to 'submission.csv'")
